In [77]:
import numpy as np
import pandas as pd
import os, re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, TweetTokenizer, RegexpTokenizer

In [63]:
flist = os.listdir(r'C:\Users\chhot\Dropbox\BlackCoffer\StopWords')

with open('StopWords.txt', 'w') as outfile:
    for file in flist:
        with open(file) as infile:
            for line in infile:
                line = line.split(' | ')[0]+'\n'
                outfile.write(line)

In [64]:
df = pd.read_table('StopWords.txt', header=None, skip_blank_lines=True, encoding='windows-1252', names=['SW'])
StopWords = df['SW'].tolist()

In [65]:
df = pd.read_excel('Output Data Structure.xlsx')
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,147,https://insights.blackcoffer.com/the-future-of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,148,https://insights.blackcoffer.com/big-data-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,149,https://insights.blackcoffer.com/business-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
def complex_word_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiou"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count += 1
    if count == 0:
        count += 1
    return count

In [67]:
path1 = r'C:\Users\chhot\Dropbox\BlackCoffer\text files'
path2 = r"C:\Users\chhot\Dropbox\BlackCoffer\text file cleaned"

tfiles = os.listdir(path1)
wrdcnt = []
cwrdcnt = []
twac = []
for i in df['URL_ID'].values:
    with open(r'{}\{}.{}'.format(path1,i,'txt'),encoding='utf-8') as infile,open(r'{}\{}.{}'.format(path2,i,'txt'),'w',encoding='utf-8') as outfile:
        text = word_tokenize(infile.read())
        wrdcnt.append(len(text))
        cwcnt=0
        for wrd in text:
            if complex_word_count(wrd)>2:
                cwcnt+=1
        cwrdcnt.append(cwcnt)
        filtext = [txt for txt in text if not txt in StopWords]
        outfile.write(' '.join(filtext))
        twac.append(len(filtext))

df['wrdcnt'] = wrdcnt
df['COMPLEX WORD COUNT'] = cwrdcnt
df['twac'] = twac

In [68]:
path3 = r"C:\Users\chhot\Dropbox\BlackCoffer\MasterDictionary" 
path4 = r"C:\Users\chhot\Dropbox\BlackCoffer\MasterDictionaryCleaned"

dfiles = os.listdir(path3)
for dfile in dfiles:
    with open(r'{}\{}'.format(path3,dfile), encoding='windows-1252') as infile, open(r'{}\{}'.format(path4,dfile), 'w', encoding='windows-1252') as outfile:
        text1 = word_tokenize(infile.read())
        filtext1 = [txt for txt in text1 if not txt in StopWords]
        outfile.write(' '.join(filtext1))

In [69]:
with open(r'{}\{}'.format(path4,'positive-words.txt'), encoding='windows-1252') as file:
    PositiveWords = word_tokenize(file.read())

with open(r'{}\{}'.format(path4,'negative-words.txt'), encoding='windows-1252') as file:
    NegativeWords = word_tokenize(file.read())

In [70]:
pscore = []
for i in df['URL_ID'].values:
    ps = 0
    with open(r'{}\{}.{}'.format(path2,i,'txt'), encoding='utf-8') as f:
        for txt in word_tokenize(f.read()):
            for txt1 in PositiveWords:
                if txt == txt1: ps+=1
    pscore.append(ps)
df['POSITIVE SCORE'] = pscore

In [71]:
nscore = []
for i in df['URL_ID'].values:
    ns = 0
    with open(r'{}\{}.{}'.format(path2,i,'txt'), encoding='utf-8') as f:
        for txt in word_tokenize(f.read()):
            for txt1 in NegativeWords:
                if txt == txt1: ns-=1
    nscore.append(-ns)
df['NEGATIVE SCORE'] = nscore

In [72]:
df['POLARITY SCORE']=round((df['POSITIVE SCORE']-df['NEGATIVE SCORE'])/((df['POSITIVE SCORE']+df['NEGATIVE SCORE'])+0.000001),2)
df['SUBJECTIVITY SCORE']=round((df['POSITIVE SCORE']+df['NEGATIVE SCORE'])/((df['twac'])+0.000001),2)

In [73]:
sentcnt = []
for i in df['URL_ID'].values:
    with open(r'{}\{}.{}'.format(path1,i,'txt'),encoding='utf-8') as infile:
        sent = sent_tokenize(infile.read())
        sentcnt.append(len(sent))
        
df['sentcnt'] = sentcnt        
df['AVG SENTENCE LENGTH']=round(df['wrdcnt']/df['sentcnt'],2)
df['PERCENTAGE OF COMPLEX WORDS']=round(df['COMPLEX WORD COUNT']/df['wrdcnt'],2)
df['FOG INDEX']=round(0.4*(df['AVG SENTENCE LENGTH']+df['PERCENTAGE OF COMPLEX WORDS']),2)

In [74]:
df['AVG NUMBER OF WORDS PER SENTENCE']=round(df['wrdcnt']/df['sentcnt'],2)

In [76]:
NLTK_StopWords = list(set(stopwords.words('english')))

tfiles = os.listdir(path1)
personal_nouns = []
for i in df['URL_ID'].values:
    with open(r'{}\{}.{}'.format(path1,i,'txt'),encoding='utf-8') as f:
        text2 = word_tokenize(f.read())
        filtext2 = [txt for txt in text2 if not txt in NLTK_StopWords and txt not in ['I', 'we','my', 'ours', 'us', 'US']]
        personal_nouns.append(len(filtext2))

df['PERSONAL PRONOUNS'] = personal_nouns       

In [82]:
personal_nouns = []
for i in df['URL_ID'].values:
    with open(r'{}\{}.{}'.format(path1,i,'txt'),encoding='utf-8') as f:
        tokens = RegexpTokenizer('[\w]+')
        tokenizer.tokenize('I', 'we','my', 'ours', 'us', 'US') 
        for 

SyntaxError: 'return' outside function (3510431610.py, line 6)

In [15]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiou"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count += 1
    if word.endswith(('es','ed')):
        count -= 1
    if count == 0:
        count += 1
    return count

In [16]:
NLTK_StopWords = list(set(stopwords.words('english')))

tfiles = os.listdir(path1)
WrdCnt = []
for i in df['URL_ID'].values:
    with open(r'{}\{}.{}'.format(path1,i,'txt'),encoding='utf-8') as f:
        text2 = word_tokenize(f.read())
        filtext2 = [txt for txt in text2 if not txt in NLTK_StopWords and txt not in ['?','!',',','.']]
        WrdCnt.append(len(filtext2))

df['WORD COUNT'] = WrdCnt

In [17]:
scl = []
for i in df['URL_ID'].values:
    with open(r'{}\{}.{}'.format(path1,i,'txt'),encoding='utf-8') as f:
        words = word_tokenize(f.read())
        sc = 0
        for word in words:
            sc += syllable_count(word)
    scl.append(sc)

df['SYLLABLE PER WORD'] = scl/df['WORD COUNT'] 

In [18]:
df.drop(columns=['twac','wrdcnt','sentcnt'], inplace=True)

In [19]:
awl = []
for i in df['URL_ID'].values:
    with open(r'{}\{}.{}'.format(path1,i,'txt'),encoding='utf-8') as f:
        words = word_tokenize(f.read())
        avgwrdlgth = 0
        for word in words:
            arpitviansh = len(word)
            avgwrdlgth = avgwrdlgth + arpitviansh
    awl.append(avgwrdlgth)
df['AVG WORD LENGTH'] = (len(word))/(len(words))

In [60]:
df.head(5)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,63,33,0.31,0.08,16.29,0.39,6.67,16.29,487,1076,2.504647,1254,0.001534
1,38,https://insights.blackcoffer.com/what-if-the-c...,56,38,0.19,0.11,10.67,0.28,4.38,10.67,239,669,2.375187,856,0.001534
2,39,https://insights.blackcoffer.com/what-jobs-wil...,65,35,0.30,0.09,13.28,0.39,5.47,13.28,443,944,2.557203,1125,0.001534
3,40,https://insights.blackcoffer.com/will-machine-...,56,24,0.40,0.09,9.87,0.28,4.06,9.87,257,747,2.369478,907,0.001534
4,41,https://insights.blackcoffer.com/will-ai-repla...,50,23,0.37,0.07,13.18,0.31,5.40,13.18,336,917,2.343511,1087,0.001534
